## PET-CT Lymphoma Segmentation with nnUNet

This notebook demonstrates how to use nnUNet for PET-CT Lymphoma Segmentation. We will use the nnUNetV2Runner MONAI implmentation to simplify the training process.

In [ ]:
import os
import random
from monai.bundle.config_parser import ConfigParser
from monai.apps.nnunet import nnUNetV2Runner

from pathlib import Path
import json

In [ ]:
dataroot = "Data"

In [ ]:
test_dir = os.path.join(dataroot,"Task100_AutoPET_Lymphoma", "imagesTs/")
train_dir = os.path.join(dataroot,"Task100_AutoPET_Lymphoma", "imagesTr/")
label_dir = os.path.join(dataroot,"Task100_AutoPET_Lymphoma", "labelsTr/")

In [ ]:
datalist_json = {"testing": [], "training": []}

In [ ]:
datalist_json["testing"] = [
    {"image": "./imagesTs/" + file}
    for file in os.listdir(test_dir)
    if ("_0000.nii.gz" in file) and ("._" not in file)
]

In [ ]:
datalist_json["training"] = [
{"image": "./imagesTr/" + file, "fold": 0, "label": "./labelsTr/" + file.replace("_0000",""),}    
for file in os.listdir(train_dir)
    if (".nii.gz" in file) and ("._" not in file)
]  # Initialize as single fold

In [ ]:
random.seed(42)
random.shuffle(datalist_json["training"])

In [ ]:
num_folds = 5
fold_size = len(datalist_json["training"]) // num_folds
for i in range(num_folds):
    for j in range(fold_size):
        datalist_json["training"][i * fold_size + j]["fold"] = i

In [ ]:
datalist_file = Path(dataroot).joinpath("Task100_AutoPET_Lymphoma","Task100_AutoPET_Lymphoma_folds.json")
with open(datalist_file, "w", encoding="utf-8") as f:
    json.dump(datalist_json, f, ensure_ascii=False, indent=4)
print(f"Datalist is saved to {datalist_file}")

In [ ]:
nnunet_root_dir = os.path.join(dataroot, "nnUNet")

os.makedirs(nnunet_root_dir, exist_ok=True)

data_src_cfg = os.path.join(nnunet_root_dir, "data_src_cfg.yaml")
data_src = {
    "modality": ["CT","PET"],
    "dataset_name_or_id": "100",
    "datalist": os.path.join(dataroot, "Task100_AutoPET_Lymphoma/Task100_AutoPET_Lymphoma_folds.json"),
    "dataroot": os.path.join(dataroot, "Task100_AutoPET_Lymphoma"),
}

ConfigParser.export_config_file(data_src, data_src_cfg)

In [ ]:
runner = nnUNetV2Runner(
    input_config=data_src_cfg, trainer_class_name="nnUNetTrainer", work_dir=nnunet_root_dir
)

In [ ]:
runner.convert_dataset()

In [ ]:
runner.plan_and_process(c= ["3d_fullres"],n_proc=[1],verify_dataset_integrity=True,npfp=1, verbose=True)